In [1]:
import pandas as pd

import os
import sys
import wave
import copy
import math
from keras.models import load_model
import tensorflow as tf


from Code.Misc.helper import *
from Code.Misc.features import *
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt
import time
from tkinter import TclError
import threading

# use this backend to display in separate Tk window
%matplotlib tk
#%matplotlib inline

# constants
TIME_SEC = 2
RATE = 44100                 # samples per second
CHUNK = int(44100 * TIME_SEC)     # 1024 * 20 #int(44100 / TIME_SEC)  # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
DEVICE = 1

# p = pyaudio.PyAudio()
# for i in range(p.get_device_count()):
#     print(p.get_device_info_by_index(i))

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [2]:
def plot_line_graph(graph, data):
    graph.axhline(y=0.5, color='r', linestyle='--')
    print(data.index)

    ang_data = data['ang']
    exc_data = data['exc']
    neu_data = data['neu']
    sad_data = data['sad']

    graph.plot(ang_data, label="Anger", color='r')
    graph.plot(exc_data, label="Excited", color='y')
    graph.plot(neu_data, label="Neutral", color='g')
    graph.plot(sad_data, label="Sad", color='b')
    graph.legend(loc="upper left")

    return graph

def plot_bar_graph (graph, data):
    graph.axhline(y=0.5, color='r', linestyle='--')
    graph.bar(x=0, height=data[0], label="Anger", color='r')
    graph.bar(x=1, height=data[1], label="Excited", color='y')
    graph.bar(x=2, height=data[2], label="Neutral", color='g')
    graph.bar(x=3, height=data[3], label="Sad", color='b')
    graph.legend(loc="upper left")

    return graph

def plot_text(text_array):
    char_size = 12
    empty_string_size = char_size
    total_input_string_size = 0

    word_index = 0.012
    text_total= ""

    plt.cla()

    plt.rcParams.update({'font.size': char_size})

    for text in text_array:
        text = text + " "
        text_size = len(text) * 0.012
        total_input_string_size += text_size

        plt.text(word_index, 0.5, text, bbox=dict(facecolor='red', alpha=0.5))
        word_index += text_size

    #plt.text(0.1, 0.5, text_total, bbox=dict(facecolor='red', alpha=0.5))

    plt.xlim([0, word_index])

    return plt

def plot_text_manual(text_array, delay_lim, delay):

    total_time = 0
    text_out = ""
    for word, time in text_array:
        total_time += 1
        if total_time > delay:
            break
        elif total_time > delay_lim:
            text_out += text

    plt.text(0.2, 0.5, text_out) #, bbox=dict(facecolor='red', alpha=0.5))

    return plt, total_time

def noise_filter(prev_val: pd.DataFrame, cur_val: pd.DataFrame, a = 0.8):

    filtered_value = pd.DataFrame()
    for col in prev_val.columns.values:
        filtered_value[col] = [a * float(prev_val[col]) +  (1-a) * float(cur_val[col])]


    print("----------")
    print("prev")
    print(prev_val)
    print("cur")
    print(cur_val)
    print("norm")
    print(filtered_value)

    return filtered_value

In [3]:
# create matplotlib figure and axes
#fig, ax = plt.subplots(1, figsize=(15, 7))

# pyaudio class instance
# p = pyaudio.PyAudio()
#
# print("starting stram decleration")
#
# def open_mic():
#     # stream object to get data from microphone
#     stream = p.open(
#         format=FORMAT,
#         channels=CHANNELS,
#         rate=RATE,
#         input=True,
#         output=True,
#         frames_per_buffer=CHUNK,
#         input_device_index = DEVICE
#     )
#
#     print("Mic open")

In [4]:
# set up model
model = load_model("../Misc/Saved_data/Trained_models/2_layer_LSTM.pickle")
cols = ['ang', 'exc', 'neu', 'sad']
df_pred_wav = pd.DataFrame([np.zeros(4)], columns=cols)
data_prev = []
data_int = []
graph_window = 30

In [5]:
# pyaudio class instance
p = pyaudio.PyAudio()

#to store np array of input data from mic
# data_int = []

def open_mic():
    print("Mic open")
    # stream object to get data from microphone
    stream = p.open(
        format=FORMAT,
        channels=CHANNELS,
        rate=RATE,
        input=True,
        output=True,
        frames_per_buffer=CHUNK,
        input_device_index = DEVICE
    )

    time_collect = time.time()

    # binary data
    data_new = stream.read(CHUNK)

    time_collect = time.time() - time_collect

    # predict_time_start = time.time()
    #convert data to integers, make np array, then offset it by 127
    data_new = struct.unpack(str(2 * CHUNK) + 'B', data_new)

    buffer.append(data_new)

    # data_int = []
    for i in range(WINDOW_N):
        data_prev = buffer[i]
        data_int = np.append(data_int, data_prev)
   #data_int = data_new

mic = threading.Thread(target=open_mic).start()

Mic open


In [6]:
# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)
fig , ax = plt.subplots(3)

# basic formatting for the axes
ax[0].set_title('Emotion Prediction')
ax[0].set_xlabel('Time')
ax[0].set_ylabel('Confidence')

ax[0]=  plot_line_graph(ax[0], df_pred_wav)
ax[1] = plot_bar_graph(ax[1], [0,0,0,0])

# show the plot
plt.show(block=False)

# for measuring frame rate
frame_count = 0
start_time = time.time()

RangeIndex(start=0, stop=1, step=1)


In [7]:
from collections import deque
WINDOW_SIZE = 0.5
WINDOW_N = 20
buffer = deque(maxlen=WINDOW_N)

CHUNK = int(RATE * WINDOW_SIZE)

#fill buffer
for i in range(WINDOW_N):
    buffer.append([])

delay = 0

# join thread
mic.join()
print("starting loop")
while (True):

    # time_collect = time.time()
    #
    # # binary data
    # data_new = stream.read(CHUNK)
    #
    # time_collect = time.time() - time_collect
    #
    # predict_time_start = time.time()
    # #convert data to integers, make np array, then offset it by 127
    # data_new = struct.unpack(str(2 * CHUNK) + 'B', data_new)
    #
    # buffer.append(data_new)
    #
    # data_int = []
    # for i in range(WINDOW_N):
    #     data_prev = buffer[i]
    #     data_int = np.append(data_int, data_prev)
   #data_int = data_new

    #Test if thread started, start timer
    print(threading.active_count())
    predict_time_start = time.time()

    # Generate features from data
    st_features = calculate_features(data_int, RATE, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=100)

    # reshape input from (34, 100) to (1, 100, 34)
    st_features = np.array([st_features.T])

    # predict on model
    with tf.device('/gpu:0'):
        wav_test_results = model.predict(st_features)

    predict_time_total = time.time() - predict_time_start

    graphing_time_start = time.time()

    predicted_values = pd.DataFrame({cols[0]:wav_test_results[0][0],
                                 cols[1]:wav_test_results[0][1],
                                 cols[2]:wav_test_results[0][2],
                                 cols[3]:wav_test_results[0][3]
    }, index=[1])

    # pass previous values to filter function
    predicted_values = noise_filter(df_pred_wav.tail(1), predicted_values)

    df_pred_wav = df_pred_wav.append(predicted_values,  ignore_index=True)
    #
    # df_pred_wav = df_pred_wav.append({cols[0]:wav_test_results[0][0],
    #                     cols[1]:wav_test_results[0][1],
    #                     cols[2]:wav_test_results[0][2],
    #                     cols[3]:wav_test_results[0][3]
    # }, ignore_index=True)

    df_pred_wav_view = df_pred_wav.tail(graph_window)
    df_pred_wav_view.reset_index(drop=True, inplace=True)
    ax[0].cla()
    ax[1].cla()

    ax[0] = plot_line_graph(ax[0], df_pred_wav_view)

    last_res = wav_test_results[0]
    ax[1] = plot_bar_graph (ax[1], last_res)
    ax[1] = plot_bar_graph (ax[1], last_res)

    graphing_time_total = time.time() - graphing_time_start
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1

    except TclError:

        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)

        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        print('prediction_time = {f} seconds'.format(predict_time_total))
    data_prev = data_new

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\walle\anaconda3\envs\TFGPU\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\walle\anaconda3\envs\TFGPU\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-afe4fa65fb23>", line 36, in open_mic
UnboundLocalError: local variable 'data_int' referenced before assignment


AttributeError: 'NoneType' object has no attribute 'join'